In [ ]:
pip install pygeos

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn
import pandas as pd
import geopandas as gp
import holoviews as hv
import geoviews as gv
hv.extension('bokeh')

In [ ]:
df = pd.read_csv('../input/nyctaxi2016/yellow_tripdata_2016-06.csv',
                parse_dates=['tpep_pickup_datetime'])
#sorteddf = df.sort_values(by='tpep_pickup_datetime')

In [ ]:
df = df.set_index('tpep_pickup_datetime')

In [ ]:
df = df.loc['2016-06-06':'2016-06-12'].sort_index()

In [ ]:
boroughs = gp.read_file('../input/nycboroughshapes/geo_export_fa106c83-ef69-4de5-8de7-5e024f12d2b4.shp')
zones = gp.read_file('../input/nycboroughshapes/taxi_zones.shp')
zones = zones[zones['borough']=='Manhattan'].to_crs('EPSG:4326')
#boroughs = boroughs.to_crs('EPSG:2263')
#boroughs

In [ ]:
plons = df['pickup_longitude'].values
plats = df['pickup_latitude'].values
dlons = df['dropoff_longitude'].values
dlats = df['dropoff_latitude'].values

times = df.index.to_series()
del df

In [ ]:
ppoints = gp.GeoDataFrame(geometry=gp.points_from_xy(plons,plats,crs='EPSG:4326'))

dpoints = gp.GeoDataFrame(geometry=gp.points_from_xy(dlons,dlats,crs='EPSG:4326'))


In [ ]:
pickdf = gp.sjoin(zones[['zone','geometry']],ppoints,how='right')
pickdf.drop(['index_left','geometry'],axis=1,inplace=True)

In [ ]:
dropdf = gp.sjoin(zones[['zone','geometry']],dpoints,how='right')
dropdf.drop(['index_left','geometry'],axis=1,inplace=True)

In [ ]:
df = pickdf.join(dropdf,lsuffix='_pick',rsuffix='_drop')

In [ ]:
df['times'] = times.values

In [ ]:
df['hour']=df['times'].dt.hour

In [ ]:
drop = df.groupby(['hour','zone_pick']).zone_drop.value_counts()

In [ ]:
(drop*drop).sum()/(drop.sum())

In [ ]:
dens_t = (drop*drop).sum(level='hour')/drop.sum(level='hour')

In [ ]:
dens_od = (drop*drop).sum(level=['zone_pick','zone_drop'])/drop.sum(level=['zone_pick','zone_drop'])

In [ ]:
dens_o = (drop*drop).sum(level='zone_pick')/drop.sum(level='zone_pick')

In [ ]:
dens_t.plot.bar(ylabel='ride density').figure.savefig('dens_t_bar.png')

In [ ]:
f1,a1 = plt.subplots()
f2,a2 = plt.subplots()
zones.merge(dens_o,left_on='zone',right_index=True,how='left').plot(column='zone_drop',legend=True,ax=a1)
f1.savefig('dens_o_map.png')
dens_o.plot.bar(ylabel='ride density',ax=a2)
f2.savefig('dens_o_bar.png')

In [ ]:
for z in ['Upper East Side South','Battery Park','Morningside Heights']:
    f1,a1 = plt.subplots()
    f2,a2 = plt.subplots()
    zones.merge(dens_od[z],left_on='zone',right_index=True,how='left').plot(column='zone_drop',legend=True,ax=a1)
    f1.savefig(f'dens_od_map_{z.replace(" ","_")}.png')
    dens_od[z].plot.bar(ylabel='ride density',ax=a2)
    f2.savefig(f'dens_od_bar_{z.replace(" ","_")}.png')